| Nome e cognome      | Insegnamento | Anno Accademico    |
| :---        |    :----   |          ---: |
| Alessandra Sirghe      | Prova finale       | 2024/2025   |


# “Analisi computazionale diacronica dell’evoluzione semantica di parole legate all’esperienza femminile nell’area anglofona (XVIII–XX secolo) tramite Word Embeddings” (Notebook 1)

## Presentazione

Il progetto che si presenta a continuazione fa parte di un tentativo più ampio di inserire gli [*Word Embeddings*](https://it.wikipedia.org/wiki/Word_embedding) all'interno di un'analisi diacronica di possibili cambi di significato riguardanti parole legate al mondo femminile e la loro contestualizzazione nei due periodi storici 1700-1832 e 1832-1932. In altre parole, si useranno spazi e rappresentazioni vettoriali di parole che permetteranno un'osservazione su larga scala (migliaia di testi) di associazioni semantiche secondo l'ipotesi della [semantica distribuzionale](https://it.wikipedia.org/wiki/Semantica_distribuzionale). Le principali aree geografiche di riferimento per il reperimento dei testi sono il Regno Unito e gli Stati Uniti (definibili come tali, ovviamente, solo successivamente alla loro effettiva costituzione), ma con ciò non va inteso che siano state usate esclusivamente opere redatte da autori provenienti dalle stesse: sono state incluse anche traduzioni di opere originariamente scritte in lingue diverse dall'inglese. I due secoli scelti, infine, rappresentano rispettivamente il periodo successivo alla Restaurazione e il Romanticismo, e l'età vittoriana con l'inizio del ventesimo secolo; il diciassettesimo è stato escluso per questioni sia pratiche (maggiore instabilità ortografica della lingua, che iniziò a normalizzarsi proprio in quel secolo) sia perché c'era la volontà di fare riferimento a queste due particolari epoche di fermento economico e sociale per rilevarne eventuali elementi di transizione e/o di distacco.

Per quel che riguarda tipologie testuali e generi, la dicotomia maschile/femminile e l'identità femminile si sono manifestati tramite rappresentazioni sia in ambito letterario sia filosofico, sociologico, teologico, ecc. lasciando quindi ampio spazio di scelta. Essendo queste le premesse ed essendo il progetto esplorativo nelle intenzioni (sono stata guidata principalmente dalla curiosità per il tipo di analisi e le possibili applicazioni di *Word Embeddings* nella ricerca del cambio semantico), non ho previsto filtri o criteri per una loro precisa selezione.  

---

Per la creazione degli *Word Embeddings* verranno allenati due modelli di [*representation learning*](https://en.wikipedia.org/wiki/Feature_learning), uno per ciascun periodo preso in considerazione dall'analisi, sfruttando la [libreria Python di **fastText**](https://fasttext.cc/docs/en/python-module.html) e la possibilità che essa dà di poter creare rappresentazioni vettoriali dense e dalle dimensioni contenute, seppur a partire da corpora particolarmente grandi.

In questo notebook, in particolare, verranno affrontate solo le fasi di reperimento dei testi e di creazione dei due corpora. Si è scelto di usare come fonte la biblioteca online del [*Project Gutenberg*](https://www.gutenberg.org/), che contiene più di 75.000 testi digitalizzati a consultazione e download gratuiti. I temi di tali opere variano dalla storia alla filosofia, dalle scienze sociali agli hobby, dalla salute alla religione; sono ovviamente presenti anche i classici della letteratura mondiale. La parte principale della collezione è costituita da opere per le quali il *copyright* risulta già scaduto negli Stati Uniti. La sua durata, che in Italia è di 70 anni, era stata presa in considerazione fin dal momento di selezione dei criteri con cui selezionare i testi.

I dettagli sullo scopo e sui temi concreti della successiva analisi verranno esposti nel secondo notebook dedicato al progetto, nel quale si è deciso di includere anche l'allenamento e la valutazione vera e propria del funzionamento dei modelli ottenuti.  

## 0. Installazione e import dei moduli/librerie necessari

In [1]:
from google.colab import drive #lo includo lo stesso, anche se ho notato che, riavviando il codice/riattivando la sessione, si installava automaticamente.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import os
import re
import glob

In [ ]:
!pip install -q gutenberg-cleaner

In [4]:
from gutenberg_cleaner import simple_cleaner, super_cleaner

In [5]:
import csv
from collections import Counter
from io import StringIO
from io import BytesIO
from pathlib import Path

import requests
import gzip #dopo una prova con solo requests, ho scoperto che nel mio caso non decomprimeva automaticamente il file che mi serviva. Da qui l'import di questa libreria

import logging
import zipfile


In [6]:
!pip install -q spacy

In [7]:
import spacy

In [8]:
nlp = spacy.blank("en")

nlp.max_length = 100000000

nlp.Defaults.stop_words |= {"w", "e", "s", "n", "m", "th", "l", "ll", "i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x", "get","go", "chap", "chapter", "gutenberg", "pg", "etexts", "texts", "project", "file", "files"}

In [9]:
!pip install nltk

In [10]:
import nltk

In [11]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

## 1. Ottenimento dei testi per i corpora

### 1.1 Reperimento metadati funzionale all'ottenimento dei testi dal *Project Gutenberg*

I testi per formare i due corpora (uno per periodo analizzato) sono stati raccolti usando due metodi differenti: quelli del secondo periodo (1832-1932) sono stati ottenuti grazie all'uso iniziale di una [query SPARQL](https://w.wiki/DrX4) effettuata su Wikidata con la quale sono state ricercate le opere letterarie pubblicate in questo intervallo di tempo e la cui lingua originale è l'inglese; per il periodo 1700-1832, invece, per la scarsità di risultati ottenuti tramite la stessa query e cambiando unicamente l'intervallo di tempo, ho seguito un [tutorial](https://skeptric.com/gutenberg/) presente all'interno del blog dell'[utente GitHub Edward Ross](https://github.com/EdwardJRoss). In esso si propone un processo di reperimento e download dei testi analogo a quello di una [libreria disponibile per il linguaggio R](https://github.com/ropensci/gutenbergr) che rispetti le linee guida del *Project Gutenberg* (*PG*) in materia di scraping e download automatico di grandi quantità di testi.

Di entrambi verranno proposte valutazioni personali sull'efficienza, facilità e quantità di risultati.

### 1.1.1 Uso di query SPARQL e Wikidata per ottenere gli ID e metadati delle opere disponibili nel *PG*

Seguendo quanto detto sopra, per reperire i testi del secondo periodo preso in esame ho usato [una query su Wikidata](https://w.wiki/DrX4) per ottenere direttamente gli identificativi dei testi pubblicati da autori di madrelingua inglese in un determinato intervallo di tempo. Propongo qui di seguito uno screenshot della query:

<img src="https://drive.google.com/uc?export=view&id=1Yk-ETNrM-bFevg1u--QL5n-i3DxHkhaT" width="800" alt="Immagine della query SPARQL usata per i testi del periodo 1832-1932"/>

L'intenzione dietro alla scelta di Wikidata era quella di rispettare dei criteri che mi ero imposta inizialmente sulla lingua originale delle opere e l'area di provenienza dei loro autori. Ho progressivamente allentato tali criteri, che nella query si presentano tramite questi passaggi:


*   Selezione delle istanze di "opera letteraria" e identificazione delle stesse tramite il loro titolo in inglese;
*   Si specifica che l'opera è dotata di autore (da identificare con il suo nome in inglese) e di un identificativo del *PG*;
*   Si specifica che la lingua in cui l'opera è scritta è l'inglese (non si tratta, in questo caso, della lingua originale);
*   Si specifica il range dentro il quale devono figurare le date di pubblicazione delle opere;
*   Gli esemplari delle opere con lo stesso titolo vengono raggruppati (non è necessario che abbiano lo stesso ID, v. più sotto la fase di pulizia delle tabelle ottenute).

Fin dal principio non era esclusa in ogni caso una possibile combinazione dei metodi di ottenimento dei testi (e, di conseguenza, un allentamento degli stessi criteri con cui sarebbero stati scelti) per entrambi i periodi, soprattutto se il numero dei risultati avesse dovuto rivelarsi maggiore per uno dei due.

Ciò mi ha permesso di scaricare e usare i risultati ottenuti in formato CSV, il che si sarebbe reso utile per eventualmente confrontarli con il catalogo del *PG*, disponibile nello stesso formato. In entrambi i casi e con procedimenti leggermente diversi, l'obiettivo era quello di poter associare le opere, in maniera diretta o indiretta, ai rispettivi ID di cui sono dotati sul sito e che appaiono nel link di riferimento a ciascuna:

<img src="https://drive.google.com/uc?export=view&id=1oEvW6XXhvHb0p4km7Uctb9FtYwW8VpN7" width="700" alt="Immagine della query SPARQL usata per i testi del periodo 1832-1932"/>


### 1.1.2 Pulizia tabelle CSV

In questa sezione si effettuerà un'operazione di pulizia della tabella legata al secondo periodo. Per farla ho scelto di usare la libreria [**pandas**](https://pandas.pydata.org/docs/).

Come si può notare dall'immagine qui sotto, alcuni degli ID del *Project Gutenberg* appaiono duplicati perché sono spesso singoli racconti contenuti all'interno di una raccolta con la quale condividono, appunto, l'ID. Inoltre, appaiono più volte anche alcuni dei titoli, indipendentemente dal fatto che abbiano lo stesso ID dei loro omonimi. Una cosa interessante al riguardo l'ho scoperta navigando nel sito del *PG*, dove si nota come le opere pubblicate in volumi siano divise negli stessi e contrassegnate da degli ID che vanno in ordine crescente di +1 (per esempio, [i due volumi in cui è diviso *Marius the Epicurean* di Walter Pater](https://www.gutenberg.org/ebooks/search/?query=marius+the+epicurean&submit_search=Go%21) hanno, rispettivamente, gli ID 4057 e 4058). Essendo questi pochi, ho contrassegnato manualmente con Excel i volumi di una stessa opera, mentre per i duplicati restanti di titolo o ID mi affiderò a pandas.

Un'ultima precisazione: essendomi utile per una delle fasi successive, ho anche cambiato il titolo della colonna "IDGutenberg" in "Text#" usando un semplice editor di testo.

<img src="https://drive.google.com/uc?export=view&id=1gABuEKlDMao260tqAdJ0VLhcZjFbR3uK" width="800" alt="Immagine della tabella CSV con gli ID duplicati evidenziati"/>


*Immagine della tabella CSV con gli ID duplicati evidenziati*





In [12]:
second_period_csv = pd.read_csv("/content/drive/MyDrive/Elaborato_finale/CSVs/1832-1932.csv") #da csv a DataFrame di pandas

In [13]:
print(f"Record iniziali della tabella:{len(second_period_csv)}")
second_period_csv.drop_duplicates(subset = "Text#", keep = "first", inplace = True) #colonna IDGutenberg: cerca valori duplicati e fai in modo di tenerne solo uno
second_period_csv.drop_duplicates(subset = "titolo", keep = "first", inplace = True) #colonna titolo: trova duplicati ed elimina uno dei due esemplari
print(f"Record rimasti dalla pulizia della tabella:{len(second_period_csv)}")

Record iniziali della tabella:777
Record rimasti dalla pulizia della tabella:753


Per controllare il risultato, potrebbe essere utile ricercare con i filtri nel DataFrame risultante gli ID segnati in rosso nell'immagine precedente.

L'ultimo passaggio di questa fase sarà salvare il DF come CSV in una cartella appositamente creata su Google Drive.

In [14]:
second_period_csv_save = second_period_csv.to_csv("/content/drive/MyDrive/Elaborato_finale/CSVs/1832-1932_cleaned.csv", index = False)

### 1.2 Ottenimento dei testi: 1832-1932

In [ ]:
second_period_df = pd.read_csv("/content/drive/MyDrive/Elaborato_finale/CSVs/1832-1932_cleaned.csv")

La fase del vero e proprio ottenimento dei testi si svolgerà seguendo le spiegazioni e buona parte del codice presente in [questa pagina](https://skeptric.com/gutenberg/) del blog dell'[utente GitHub Edward Ross](https://github.com/EdwardJRoss). L'unico cambio sostanziale che ho fatto è stato nella fase di lavoro con [DictReader](https://docs.python.org/3/library/csv.html#csv.DictReader), dove ho preferito usare pandas per poter unire il CSV del catalogo completo del *PG* con quello degli ID raccolti con Wikidata per il secondo intervallo di tempo tramite il metodo **merge**. Ulteriori cambiamenti in conseguenza di ciò verranno specificati nei successivi blocchi di documentazione.

Il codice qui di seguito presenta, inoltre, due linee commentate in inglese suggerite da Gemini per poter attuare la trasformazione della stringa ottenuta con lo *scraping* di Requests in un formato convertibile in un DataFrame di pandas. Gemini ha suggerito di usare il [metodo **StringIO**](https://docs.python.org/3/library/io.html#io.StringIO) per simulare l'apertura e la lettura di un file e poterlo così leggere con il metodo **read_csv** di pandas che, da [documentazione](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html), ammette anche gli oggetti come quelli creati proprio con StringIO che, pur non essendo veri e propri file, si possono leggere con metodi analoghi a **read()**.

In [16]:
GUTENBERG_CSV_URL = "https://www.gutenberg.org/cache/epub/feeds/pg_catalog.csv.gz"

r = requests.get(GUTENBERG_CSV_URL)
csv_text = gzip.decompress(r.content).decode("utf-8")

# Use StringIO to create a file-like object from the string
csv_buffer = StringIO(csv_text)

# Now you can use read_csv on the buffer
gutenberg_full_catalogue = pd.read_csv(csv_buffer)


print(f"Total size: {len(r.content) / 1024**2:0.2f}MB")
print(f"Total number of rows: {len(gutenberg_full_catalogue)}")

Total size: 5.16MB
Total number of rows: 76602


A questo punto, unisco i CSV del catalogo completo del *PG* con quello da me ottenuto tramite la query su Wikidata con il metodo merge (applicato, ovviamente, sugli ID in comune tra i due).

In [17]:
second_period_gutenberg_books = pd.merge(second_period_df, gutenberg_full_catalogue, on = "Text#", how="inner")

Testando dei blocchi di codice più in avanti, tuttavia, ho notato che nel DataFrame (DF) risultante rimanevano dei record relativi a opere non in lingua inglese. Per risolvere quest'ultimo problema ho usato il metodo **loc** di pandas per selezionare solo quelle righe contenenti i metadati di file in lingua inglese seguendo questo tutorial su [GeeksforGeeks](https://www.geeksforgeeks.org/drop-rows-from-the-dataframe-based-on-certain-condition-applied-on-a-column/).

In [18]:
second_period_gutenberg_books = second_period_gutenberg_books.loc[second_period_gutenberg_books["Language"] == "en"]
second_period_gutenberg_books.head(90)

,titolo,Text#,Type,Issued,Title,Language,Authors,Subjects,LoCC,Bookshelves
0,The Man Who Knew Too Much,1720,Text,1999-04-01,The Man Who Knew Too Much,en,"Chesterton, G. K. (Gilbert Keith), 1874-1936","Detective and mystery stories, English; Great ...",PR,"Category: Crime, Thrillers and Mystery; Catego..."
1,The Primrose Ring,15482,Text,2005-03-27,The Primrose Ring,en,"Sawyer, Ruth, 1880-1970",Fairy tales; Medical fiction,PS,Category: Novels; Category: American Literature
2,The Man in the Brown Suit,61168,Text,2020-01-14,The Man in the Brown Suit,en,"Christie, Agatha, 1890-1976",South Africa -- Fiction; Detective and mystery...,PR,"Detective Fiction; Category: Crime, Thrillers ..."
3,The Secret Adversary,1155,Text,1998-01-01,The Secret Adversary,en,"Christie, Agatha, 1890-1976",Private investigators -- England -- Fiction; D...,PR,"Detective Fiction; Category: Crime, Thrillers ..."
4,Tarzan and the Golden Lion,58874,Text,2019-02-11,Tarzan and the Golden Lion,en,"Burroughs, Edgar Rice, 1875-1950; St. John, J....",Tarzan (Fictitious character) -- Fiction; Fant...,PS,Category: Adventure; Category: Novels; Categor...
...,...,...,...,...,...,...,...,...,...,...
86,Tom Swift and His Electric Runabout,950,Text,1997-06-01,"Tom Swift and His Electric Runabout; Or, The S...",en,"Appleton, Victor","Swift, Tom (Fictitious character) -- Juvenile ...",PZ,Children's Book Series; Category: Children & Y...
87,The Abysmal Brute,55948,Text,2017-11-12,The Abysmal Brute,en,"London, Jack, 1876-1916; Grant, Gordon, 1875-1...",Boxers (Sports) -- Fiction; Boxing stories,PS,Category: Novels; Category: American Literatur...
88,The Helmet of Navarre,14219,Text,2004-11-30,The Helmet of Navarre,en,"Runkle, Bertha, 1879-1958; Castaigne, J. André...","Henry IV, King of France, 1553-1610 -- Fiction...",PS,"Bestsellers, American, 1895-1923; Category: Hi..."
90,Wives and Daughters,4274,Text,2003-07-01,Wives and Daughters,en,"Gaskell, Elizabeth Cleghorn, 1810-1865",England -- Fiction; Young women -- Fiction; Lo...,PR,Category: Novels; Category: British Literature


In [19]:
print(len(second_period_gutenberg_books))

748


In [ ]:
#second_period_gutenberg_books.to_csv("/content/drive/MyDrive/Elaborato_finale/CSVs/1832-1932_gutenberg.csv", index = False) #salvataggio intermedio del lavoro svolto

In [20]:
second_period_gutenberg_books = pd.read_csv("/content/drive/MyDrive/Elaborato_finale/CSVs/1832-1932_gutenberg.csv")

Ho trasformato quindi il *data type* della colonna con gli ID da *integer* a stringa con il metodo **astype** e controllato che funzionasse con **dtypes**. È interessante il fatto che nella documentazione di pandas siano indicati [due tipi di dato](https://pandas.pydata.org/docs/user_guide/text.html#text-data-types) funzionali a conservare e lavorare con testi, entrambi presenti nei risultati dati da dtypes.

In [21]:
second_period_gutenberg_books["Text#"] = second_period_gutenberg_books["Text#"].astype("string")
second_period_gutenberg_books.dtypes #controllo

,0
titolo,object
Text#,string[python]
Type,object
Issued,object
Title,object
Language,object
Authors,object
Subjects,object
LoCC,object
Bookshelves,object


Da questo punto si inizia, quindi, a seguire linearmente la spiegazione di Ross: essa si basa sul funzionamento della libreria [**gutenbergr**](https://github.com/ropensci/gutenbergr), creata per essere usata con il linguaggio di programmazione R e che permette di scaricare le opere dal *PG* in accordo con le loro politiche di download automatico dei materiali accessibili dal sito.

Qui di seguito elenco i passaggi fondamentali:


1.   Creazione di un URL adatto al ritrovamento di un [*mirror*](https://en.wikipedia.org/wiki/Mirror_site) del *PG* (*aleph.gutenberg.org*) da cui si effettuerà lo *scraping*.
2.   Costruzione dell'URL con cui si andrà alla ricerca del/dei file nel *mirror*.
3.   Una prova di download con uno dei documenti contenuti nel DF appena ottenuto.
4.   Definizione di una funzione che mi permetta di ottenere il testo delle opere e di pulirli (effettuando anche una prova con una libreria apposita).
5.   Definizione di un'ulteriore funzione che permetterà di iterare tra gli ID del DF-catalogo in uso per "riscrivere" automaticamente i testi delle opere da essi identificati in altrettanti file TXT da conservare in una cartella sul mio Google Drive.



In [22]:
GUTENBERG_ROBOT_URL = "http://www.gutenberg.org/robot/harvest?filetypes[]=txt"

book_id = second_period_gutenberg_books["Text#"] #assegnazione alla variabile book_id dei contenuti della colonna "Text#" del DF ottenuto in precedenza

r = requests.get(GUTENBERG_ROBOT_URL)

print(r.text[:750]) #gli URL ottenuti verranno usati per identificare l'indirizzo del mirror

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">

<html lang="en">
  <head>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
    <title>All Files (offset: 0, filetypes: txt) - Project Gutenberg</title>
  </head>
  <body>
    <h1>All Files (offset: 0, filetypes: txt)</h1>    <p><a href="http://aleph.gutenberg.org/1/0/0/8/10084/10084-8.zip">http://aleph.gutenberg.org/1/0/0/8/10084/10084-8.zip</a></p>

    <p><a href="http://aleph.gutenberg.org/1/0/0/8/10084/10084.zip">http://aleph.gutenberg.org/1/0/0/8/10084/10084.zip</a></p>

    <p><a href="http://aleph.gutenberg.org/1/5/5/1554/1554.zip">http://aleph.gutenberg.org/1/5/5/1554/1554.zip</a></p>

    <p><a href="http://aleph.guten


In [23]:
GUTENBERG_MIRROR = re.search('(https?://[^/]+)[^"]*.zip', r.text).group(1)
GUTENBERG_MIRROR

'http://aleph.gutenberg.org'

In [24]:
def gutenberg_text_urls(id: str, mirror=GUTENBERG_MIRROR, suffixes=("", "-8", "-0")) -> list[str]: #vengono usate annotazioni per specificare il data-type degli ID e del risultato della funzione: https://peps.python.org/pep-3107/
    path = "/".join(id[:-1]) or "0"
    return [f"{mirror}/{path}/{id}/{id}{suffix}.zip" for suffix in suffixes] #i "suffissi" sono quelli identificati grazie al blocco di codice precedente

gutenberg_text_urls(book_id)

['http://aleph.gutenberg.org/1720/15482/61168/1155/58874/12037/10897/1848/8954/11639/4307/955/71065/1145/2440/22158/70844/14872/37665/3177/308/2149/1156/9846/14257/70486/26499/14471/898/58877/2305/12450/25322/560/17508/2024/10832/33043/5759/1695/56876/71268/9629/4282/700/963/60590/28948/638/15408/1227/3782/3829/786/61085/39162/3470/58664/46807/2856/24/70379/34406/958/1860/58666/58956/14579/2449/706/1193/34089/26918/60333/96/71273/59844/10806/70236/4917/2097/10095/6877/6769/60547/950/55948/14219/4274/81/21639/60093/70736/271/13707/1611/70271/215/3031/780/2002/11438/60944/14978/1093/69700/3261/19362/11051/1419/1948/68/2346/4217/4520/16517/43604/1999/63293/329/169/21249/7308/10291/43/33913/5346/217/8677/604/64329/921/201/3189/14797/21552/46709/2511/647/2726/27771/6686/10977/4240/2370/27712/3837/35117/172/769/42455/2350/7381/268/914/837/439/1487/4368/25584/70912/22704/60097/47086/678/2081/2028/86/3289/2343/2841/35517/605/10165/32242/1162/70777/68283/1721/5247/6455/30855/70152/2604/4058/690

Il blocco di codice che segue corrisponde all'azione descritta al punto numero 3 della lista precedente. Si tratta, quindi, della definizione della funzione che permette di ottenere il contenuto dei testi a partire dai link creati con la funzione gutenberg_text_urls. Rispetto al tutorial di Ross, tuttavia, ho dovuto aggiungere della parte di codice per far fronte a un errore del tipo "BadZipFile", sollevato dalla libreria **zipfile**, in cui mi si avvertiva che si stava cercando di aprire un file dal formato diverso da quello atteso (ZIP). Le aggiunte appaiono nel blocco try/except dopo il comando "break".



In [25]:
def download_gutenberg(id: str) -> str:
    for url in gutenberg_text_urls(id):
        r = requests.get(url)
        if r.status_code == 404:
            logging.warning(f"404 for {url}")
            continue
        r.raise_for_status()
        break

    try:

      z = zipfile.ZipFile(BytesIO(r.content))
      if len(z.namelist()) != 1:
            raise Exception(f"Expected 1 file in {z.namelist()}")

      return z.read(z.namelist()[0]).decode('utf-8')

    except zipfile.BadZipFile as error:
        print(f"An error has been encountered while dealing with the ZIP file: {error}")



In [26]:
book_id_to_test = second_period_gutenberg_books["Text#"][678] #prova con il testo contrassegnato dall'ID presente alla riga indicizzata con il numero n
text = download_gutenberg(book_id_to_test)

if text is not None: #anche questa è un'aggiunta resa necessaria da un errore nel momento della stampa del testo con certi ID a cui era associato un NoneType
  print(text[:1500])
else:
  print(f"Could not retrieve text for book ID {book_id_to_test}")

The Project Gutenberg EBook of Tom Grogan, by F. Hopkinson Smith

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org


Title: Tom Grogan

Author: F. Hopkinson Smith

Posting Date: July 26, 2008 [EBook #850]
Release Date: March, 1997

Language: English

Character set encoding: ASCII

*** START OF THIS PROJECT GUTENBERG EBOOK TOM GROGAN ***




Produced by An Anonymous Volunteer





TOM GROGAN

by F. Hopkinson Smith




I. BABCOCK'S DISCOVERY

Something worried Babcock. One could see that from the impatient gesture
with which he turned away from the ferry window on learning he had half
an hour to wait. He paced the slip with hands deep in his pockets, his
head on his chest. Every now and then he stopped, snapped open his watch
and shut it again quickly, as if to hurry the lagging minutes.

Fo

Successivamente a questa prova, passo al download vero e proprio di tutti i testi il cui ID è presente nel mio DF-catalogo di riferimento. Per la pulizia delle parti iniziale e finale degli stessi (caratterizzate dalla presenza di metadati e/o informazioni sulla licenza d'uso dei materiali presenti nel *PG*) ho voluto provare una libreria messa a disposizione da un utente di GitHub, [**gutenberg-cleaner**](https://github.com/kiasar/gutenberg_cleaner), oltre al codice messo a disposizione dallo stesso Ross (v. funzione **strip_headers** poco più avanti). Ho voluto fare una prova per testare le tre opzioni iniziando dai metodi **simple_cleaner** e **super_cleaner** della libreria. Quest'ultimo viene descritto non solo come più efficace ma anche come più aggressivo, in quanto può portare alla rimozione di parti delle opere sottoposte alla pulizia.

Qui di seguito si possono analizzare i rispettivi rendimenti con il testo contrassegnato dall'ID indicizzato al numero 678 del DF (v. nel blocco qui sopra la variabile book_id). In generale, posso dire che **super_cleaner** sembra avere la meglio in termini di precisione nell'individuare le parti da eliminare, come le indicazioni intermedie sull'inizio o la fine di un capitolo. Tuttavia, dopo averlo testato su varie opere, ho notato che effettivamente poteva portare all'eliminazione di materiale testuale importante, ragione per la quale ho deciso di non usarlo.

Tra la *state machine* di Ross (intuitivamente, per me, se ne può paragonare il funzionamento a quello di una funzione "a bandierina") e simple_cleaner, infine, non ho notato differenze così evidenti da giustificare l'uso dell'una piuttosto che dell'altra. Tuttavia, essendo stato il codice di Ross già testato in un caso reale, propendo per l'uso di quest'ultimo. Il risultato, come si noterà dai blocchi proposti qui di seguito, sarà ovviamente imperfetto per la presenza di parti non legate al contenuto testuale vero e proprio: se la parte finale delle opere del *PG* è abbastanza standardizzata e facile da eliminare secondo delle semplici condizioni, quella iniziale con indice e metadati è più difficile da sintetizzare con delle RegEx o, appunto, con delle condizioni altrettanto generali.   

In [27]:
GUTENBERG_TEXT = "PROJECT GUTENBERG EBOOK "

def strip_headers(text):
    in_text = False
    output = []

    for line in text.splitlines():
        if GUTENBERG_TEXT in line: #se la dicitura "PROJECT GUTENBERG EBOOK " è presente in una data riga (definita dalla presenza del separatore \n)
            if not in_text:
                in_text = True # assegna un nuovo valore "stabile" per la stessa variabile e fai iniziare il contenuto vero e proprio
            else:
                break
        else:
            if in_text:
                output.append(line) #se non è ancora stata raggiunta la seconda occorrenza di GUTENBERG_TEXT (in_text è ancora True), allora aggiungi la riga alla lista output

    return "\n".join(output).strip()

#stripped_text = strip_headers(text)

In [28]:
cleaned_text_w_simplec = simple_cleaner(text)
print(cleaned_text_w_simplec[:6000])
print("-------------------------------------------------------------------------------")
print(cleaned_text_w_simplec[-6000:])






TOM GROGAN

by F. Hopkinson Smith




I. BABCOCK'S DISCOVERY

Something worried Babcock. One could see that from the impatient gesture
with which he turned away from the ferry window on learning he had half
an hour to wait. He paced the slip with hands deep in his pockets, his
head on his chest. Every now and then he stopped, snapped open his watch
and shut it again quickly, as if to hurry the lagging minutes.

For the first time in years Tom Grogan, who had always unloaded his
boats, had failed him. A scow loaded with stone for the sea-wall that
Babcock was building for the Lighthouse Department had lain three days
at the government dock without a bucket having been swung across her
decks. His foreman had just reported that there was not enough material
to last the concrete-mixers two hours. If Grogan did not begin work at
once, the divers must come up.

Heretofore to turn over to Grogan the unloading of material for any
submarine work had been like feeding grist to a mill--so ma

In [29]:
cleaned_text_w_superc = super_cleaner(text)
print(cleaned_text_w_superc[:1500])
print("-------------------------------------------------------------------------------")
print(cleaned_text_w_superc[-6000:])

[deleted]

[deleted]

[deleted]

[deleted]

[deleted]

[deleted]

Something worried Babcock. One could see that from the impatient gesture
with which he turned away from the ferry window on learning he had half
an hour to wait. He paced the slip with hands deep in his pockets, his
head on his chest. Every now and then he stopped, snapped open his watch
and shut it again quickly, as if to hurry the lagging minutes.

For the first time in years Tom Grogan, who had always unloaded his
boats, had failed him. A scow loaded with stone for the sea-wall that
Babcock was building for the Lighthouse Department had lain three days
at the government dock without a bucket having been swung across her
decks. His foreman had just reported that there was not enough material
to last the concrete-mixers two hours. If Grogan did not begin work at
once, the divers must come up.

Heretofore to turn over to Grogan the unloading of material for any
submarine work had been like feeding grist to a mill--so man

In [31]:
stripped_text = strip_headers(text)
print(stripped_text[:1500])
print("-------------------------------------------------------------------------------")
print(stripped_text[-6000:])

Produced by An Anonymous Volunteer





TOM GROGAN

by F. Hopkinson Smith




I. BABCOCK'S DISCOVERY

Something worried Babcock. One could see that from the impatient gesture
with which he turned away from the ferry window on learning he had half
an hour to wait. He paced the slip with hands deep in his pockets, his
head on his chest. Every now and then he stopped, snapped open his watch
and shut it again quickly, as if to hurry the lagging minutes.

For the first time in years Tom Grogan, who had always unloaded his
boats, had failed him. A scow loaded with stone for the sea-wall that
Babcock was building for the Lighthouse Department had lain three days
at the government dock without a bucket having been swung across her
decks. His foreman had just reported that there was not enough material
to last the concrete-mixers two hours. If Grogan did not begin work at
once, the divers must come up.

Heretofore to turn over to Grogan the unloading of material for any
submarine work had been 

In [32]:
GUTENBERG_TEXT_URL = "https://www.gutenberg.org/ebooks/{id}.txt.utf-8" #URL che rispetta il formato necessario per identificare i singoli testi

Questa la definizione della funzione che, oltre a fare lo scraping del testo delle opere, mi permette di ripulirli.

In [33]:
def book_text(book_id):
    r = requests.get(GUTENBERG_TEXT_URL.format(id=book_id))
    text = r.text
    cleaned_text = strip_headers(text)
    return cleaned_text

Definisco infine la funzione (anche questa ripresa nella sua maggior parte dal tutorial di Ross) che mi permetterà di ottenere i testi e ripulirli con la funzione appena definita, e "riscriverli" nei corrispondenti file TXT, per i quali ho creato una cartella apposita nel mio Drive. Ho usato il modulo **os** e il metodo **os.path.join** per fare in modo che i testi, contrassegnati dal loro ID, siano scritti nella stessa.

In [49]:
def gutenberg_txts_download(df, path, id_column_name):

  if not df.empty:
    isdir = os.path.isdir(path)

    if isdir: #breve controllo sulla directory fornita come argomento

        for book_id in df[id_column_name]: #per iterare nella colonna (Series)
            file_name = str(book_id) + ".txt" #mi assicuro che il risultato sia una stringa
            path = path
            complete_path = os.path.join(path, file_name) #dove i testi (in quale file) verranno scritti
            text = book_text(book_id) #applicazione della funzione per ottenimento/pulizia dei testi
            print(f"Saving {file_name} containing {len(text):_} characters")
            #print(book_count)
            with open(complete_path, "wt") as f:
                f.write(text)

    else:
      print("Check the path you provided.")

  else:
    print("The DF you provided is empty.")

In [ ]:
gutenberg_txts_download(second_period_gutenberg_books, "/content/drive/MyDrive/Elaborato_finale/txts/1832-1932", "Text#")

Contando il numero dei file ottenuti, i risultati finali per il periodo 1832-1932 sono 748.


### 1.2.1 Conclusione fase di ottenimento dei testi per il periodo 1832-1932

Come già anticipato nel riferimento introduttivo ai diversi metodi di reperimento dei testi, visto il basso numero di risultati ottenuti per il periodo 1700-1831 si è deciso di seguire, con ben poche modifiche, [la strada suggerita dall'utente GitHub Edward Ross](https://skeptric.com/gutenberg/) nel suo blog **skeptric**. Tuttavia, portando quest'ultima ad avere un numero di risultati quantitativamente superiore anche rispetto alle 748 opere ottenute con gli ID presenti nei record di Wikidata per il periodo 1832-1932, anticipo per comodità personale (e per rendere il Notebook il più lineare possibile nella spiegazione delle fasi successive del progetto) i punti e le funzioni legate all'ottenimento dei testi del periodo 1700-1832.

Infatti, ho verificato che i due corpora fossero effettivamente sbilanciati quantitativamente tramite il conteggio dei rispettivi *tokens* (v. immagini sotto) e, per ottenerne da subito di più equilibrati ho applicato il metodo di Ross al corpus appena generato e aggiunto alla cartella su Drive un numero controllato di file TXT in più per integrarlo. Sempre per ragioni di linearità, le funzioni che seguono non verranno spiegate nel dettaglio, bensì solo in generale secondo il compito che portano a termine: riserverò le spiegazioni più approfondite per la sezione 1.3 del Notebook, la quale è dedicata proprio all'ottenimento dei testi dell'intervallo 1700-1831.


<img src="https://drive.google.com/uc?export=view&id=1qaupBAbhP62bVcBGIuLd0GHLtYMcGdpF" width="1600" alt="Immagine con numero tokens per il corpus del periodo 1700-1831"/>

*Screenshot di documentazione del numero di tokens presenti nel corpus del primo periodo (1700-1831)*

<img src="https://drive.google.com/uc?export=view&id=1sIqE5EruisUerSuFe0KWh243CnpL7ba_" width="800" alt="Immagine con numero tokens per il corpus del periodo 1832-1932"/>

*Screenshot di documentazione del numero di tokens presenti nel corpus del secondo periodo (1832-1932)*


Ai fini della documentazione del procedimento, propongo qui di seguito la funzione che mi ha permesso di contare i tokens ottenuti, dopo aver trasferito i file TXT in un DataFrame di pandas:
```
 def tokens_counter(tokenized_text_in_df_column):

    df_to_ds = tokenized_text_in_df_column.explode()
    total_number_of_words = df_to_ds.count()

    return total_number_of_words
```
L'idea era quella di usare i due metodi di pandas [**explode**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html) e [**count**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.count.html): il primo permette di separare ciascun elemento contenuto nei testi tokenizzati (che sono, letteralmente, liste di parole) in righe a sé, che poi vengono contate con il secondo metodo.

Questo conteggio aveva anticipato la fase vera e propria di *preprocessing*, che in una prima versione del progetto avevo tentato di portare a termine con pandas. Tuttavia, anticipo che in questa versione finale pandas non è stato usato per ragioni che verranno esposte più avanti.


Questa **prima funzione** serve a ottenere i record del catalogo del *PG* di opere prodotte da autori vissuti in un determinato range temporale:

In [50]:
def filter_books_by_author_lifespan(csv_catalogue, start_year, end_year):
    matching_books = []

    year_pattern = re.compile(r'(\d{4})\s*-\s*(\d{4})')  # e.g., "1775-1817"

    reader = csv.DictReader(StringIO(csv_catalogue))
    for book in reader:
        if book["Language"].strip().lower() != "en":
            continue

        author_field = book.get("Authors", "") #ritorna valore per quella chiave
        matching_string = year_pattern.search(author_field) #con search si ritorna solo la prima occorrenza del pattern
        if matching_string:
            birth_year = int(matching_string.group(1)) #con group si ottiene, qui, l'occorrenza del primo gruppo definito tra () nella RegEx
            death_year = int(matching_string.group(2)) #idem, ma si ottiene il secondo gruppo
            if death_year >= start_year and death_year <= end_year:
              if birth_year >= start_year and birth_year <= end_year:
                  matching_books.append(book)

    print(f"Found {len(matching_books)} matching books.")
    return matching_books



Questa **seconda funzione** serve, se usata con quella precedente, a creare un DataFrame che contenga un numero esatto di record relativi ai testi di quegli autori nati nel periodo definito e che non siano contenuti già nel DF da cui ho tratto le opere precedenti per lo stesso.   

In [51]:
def get_tot_number_of_records(csv_catalogue, start_year, end_year, limit, list_of_forbidden_ids):

    list_of_matching_books_records = filter_books_by_author_lifespan(csv_catalogue, start_year, end_year)
    df_of_matching_books = pd.DataFrame.from_dict(list_of_matching_books_records) #da una lista di dizionari

    if list_of_forbidden_ids:
      index_ids = df_of_matching_books[df_of_matching_books["Text#"].isin(list_of_forbidden_ids)].index
      df_of_matching_books.drop(index_ids, inplace = True)

    else:
      print("No list with possible 'forbidden' IDs found.")

    index_with_doc_type = df_of_matching_books[(df_of_matching_books["Type"] != "Text")].index
    df_of_matching_books.drop(index_with_doc_type, inplace = True)

    index_with_subjects_col = df_of_matching_books[(df_of_matching_books["Subjects"] == "Music") | (df_of_matching_books["Subjects"] == "Indexes")].index
    df_of_matching_books.drop(index_with_subjects_col, inplace = True)

    if limit:
      df_of_matching_books = df_of_matching_books.sample(n = limit, random_state = 1) #random_state = 1 permette di ottenere risultati replicabili

    else:
      print("No limit of records found.")

    print(f"Found {len(df_of_matching_books)} new books to add to your corpus!")

    return df_of_matching_books

In [52]:
list_of_ids_already_in_corpus = second_period_gutenberg_books["Text#"].to_list() #creo la lista degli ID da evitare, perché di opere già scaricate in precedenza

In [53]:
new_second_period_books_df = get_tot_number_of_records(csv_text, 1817, 1932, 450, list_of_ids_already_in_corpus)

new_second_period_books_df.head()

Found 20868 matching books.
Found 450 new books to add to your corpus!


,Text#,Type,Issued,Title,Language,Authors,Subjects,LoCC,Bookshelves
5208,15564,Text,2005-04-06,"The Philippine Islands, 1493-1898 — Volume 18 ...",en,"Blair, Emma Helen, 1851-1911 [Editor]; Bourne,...",Philippines -- History -- Sources; Missions --...,DS,Category: History - Other; Category: History -...
1442,3630,Text,2003-01-01,What to Do? Thoughts Evoked by the Census of M...,en,"Tolstoy, Leo, graf, 1828-1910; Hapgood, Isabel...",Social problems; Moscow (Russia) -- Social con...,HN,"Category: Essays, Letters & Speeches; Category..."
2539,6253,Text,2004-11-18,Michel and Angele [A Ladder of Swords] — Complete,en,"Parker, Gilbert, 1862-1932","Great Britain -- History -- Elizabeth, 1558-16...",PS,Category: Historical Novels; Category: Romance...
9828,32183,Text,2010-04-29,Folk-lore of Shakespeare,en,"Thiselton-Dyer, T. F. (Thomas Firminger), 1848...","Shakespeare, William, 1564-1616 -- Knowledge -...",PR,Category: Plays/Films/Dramas; Category: Mythol...
5167,15401,Text,2005-03-18,The Great Lone Land\nA Narrative of Travel and...,en,"Butler, William Francis, Sir, 1838-1910","Northwest, Canadian -- Description and travel",F1001,Category: Adventure; Category: Travel Writing;...


In [ ]:
gutenberg_txts_download(new_second_period_books_df, "/content/drive/MyDrive/Elaborato_finale/txts/1832-1932", "Text#")

Saving 34017.txt containing 538_104 characters
Saving 54050.txt containing 391_623 characters
Saving 27.txt containing 773_819 characters
Saving 28670.txt containing 159_924 characters
Saving 66318.txt containing 1_057_063 characters
Saving 19108.txt containing 952_038 characters
Saving 41137.txt containing 514_006 characters
Saving 37744.txt containing 460_151 characters
Saving 14627.txt containing 169_716 characters
Saving 35894.txt containing 427_040 characters
Saving 40470.txt containing 660_233 characters
Saving 9610.txt containing 92_885 characters
Saving 57331.txt containing 189_520 characters
Saving 45913.txt containing 43_277 characters
Saving 40701.txt containing 252_361 characters
Saving 15495.txt containing 279_666 characters
Saving 44412.txt containing 548_218 characters
Saving 42547.txt containing 233_288 characters
Saving 38497.txt containing 522_965 characters
Saving 446.txt containing 532_729 characters
Saving 298.txt containing 673_017 characters
Saving 3754.txt conta

<_io.TextIOWrapper name='/content/drive/MyDrive/Elaborato_finale/txts/1832-1932/34071.txt' mode='wt' encoding='utf-8'>

### 1.3 Ottenimento dei testi: 1700-1832 (1831)

Il metodo adottato per ottenere i testi del primo periodo (1700-1832) ha previsto l'uso del solo catalogo del *PG*, delle due funzioni **book_text** e **gutenberg_txts_download**, e di un'ulteriore funzione di "filtraggio" dei record dello stesso in base alla lingua di scrittura delle opere e alle date di nascita e di morte presenti accanto al primo autore che figura nella stringa che fa da valore alla chiave "Authors" (la funzione **filter_books_by_author_lifespan**, la prima tra le due definite qui sopra). Il catalogo del *PG*, infatti, viene reso come una lista di dizionari.

Il codice di questa funzione di filtraggio è stata **suggerita da ChatGPT**. Ho solo apportato un piccolo accorgimento nella parte finale e nelle condizioni riguardanti il tipo di file delle opere da trovare. Ho ritenuto anche adeguato cambiare il nome alla variabile *match* per non farlo coincidere con [altre istruzioni convenzionali di Python](https://realpython.com/structural-pattern-matching/#getting-to-know-structural-pattern-matching).

Qui di seguito la domanda che ho fatto a ChatGPT e il codice originale avuto come risposta:


> [...] how could I look up for records [nel catalogo del *PG*] whose main author has lived in between the period I am analysing?



```
# import csv
import re
from io import StringIO

def filter_books_by_author_lifespan(catalogue_csv_text, start_year, end_year):
    matching_books = []

    year_pattern = re.compile(r'(\d{4})\s*-\s*(\d{4})')  # e.g., "1775-1817"

    reader = csv.DictReader(StringIO(catalogue_csv_text))
    for book in reader:
        if book["Language"].strip().lower() != "en":
            continue

        author_field = book.get("Authors", "")
        match = year_pattern.search(author_field)
        if match:
            birth_year = int(match.group(1))
            death_year = int(match.group(2))
            if birth_year <= end_year and death_year >= start_year:
                matching_books.append(book)

    print(f"Found {len(matching_books)} matching books.")
    return matching_books
```

Come si può notare confrontandola con quella effettivamente usata, il mio cambio è stato minimo: l'unico problema che avevo trovato nel valutare il codice dell'IA era stata la possibilità di ritrovarmi tra i risultati anche record nel catalogo del *PG* che contenessero autori la cui data di morte fosse successiva all'anno indicato dalla variabile "end_year".

Un'ultima precisazione: la data d'inizio del range l'ho modificata da "1700" a "1685", tenendo in conto il fatto che si tratta di date di nascita/morte e non di date di pubblicazione: ho lasciato quindici anni di tolleranza per calcolare una possibile età di inizio carriera e produzione letteraria nei primi anni del XVIII secolo. La stessa cosa è stata fatta per estrarre i record aggiuntivi per il periodo 1832-1932.

Riutilizzo, quindi, questa funzione all'interno della seconda (**get_tot_number_of_records**), dove si crea inzialmente un DF dai risultati ottenuti con la funzione filter_books_by_author_lifespan e, se passata come argomento, si eliminano da questo i record che contengono le opere di una lista di ID "vietati" (doppioni), opere non testuali o dai soggetti poco rilevanti per il tipo di testo che possono contenere (indici, elenchi o spartiti musicali, per esempio); una volta effettuata questa fase di filtraggio, se passato come argomento alla funzione, verrà casualmente selezionato solo un certo numero di record (nel mio caso, ovviamente, ciò era voluto per controllare il numero di testi da aggiungere a quelli presenti nella cartella sul Drive). Questa volta non si specificano ID da evitare o limiti nel numero di opere da ottenere. Al suo interno, infatti, ho posto delle condizioni che mi permettono di ignorare tali argomenti se nulli.

Per il filtraggio ho usato il metodo **drop** scegliendo tra le possibilità [qui](https://www.geeksforgeeks.org/drop-rows-from-the-dataframe-based-on-certain-condition-applied-on-a-column/) descritte.





In [ ]:
first_period_df = get_tot_number_of_records(csv_text, 1685, 1831, None, None)

Found 1488 matching books.
No list with possible 'forbidden' IDs found.
No limit of records found.
Found 1412 new books to add to your corpus!


Il numero dei risultati è considerevolmente più alto rispetto a quello delle opere ottenute con la query SPARQL e Wikimedia. È quindi evidente che questo metodo, per quanto non permetta di essere precisi quanto il primo per quel che riguarda caratteristiche come la lingua originale dell'opera, nazionalità dell'autore e filtraggio per data di pubblicazione della stessa, è il migliore per quantità di risultati.

A questo punto, salvo sia i metadati sia, ovviamente, i testi (questi ultimi sempre "scritti" all'interno di una cartella nel mio Drive con la funzione **gutenberg_txts_download**).

In [ ]:
first_period_df.to_csv("/content/drive/MyDrive/Elaborato_finale/CSVs/1700-1831 (1).csv", index = False)

In [ ]:
gutenberg_txts_download(first_period_df, "/content/drive/MyDrive/Elaborato_finale/txts/1700-1831", "Text#")

Saving 1.txt containing 104_993 characters
Saving 6.txt containing 14_852 characters
Saving 18.txt containing 1_172_758 characters
Saving 105.txt containing 464_827 characters
Saving 121.txt containing 434_000 characters
Saving 134.txt containing 258_858 characters
Saving 141.txt containing 883_559 characters
Saving 147.txt containing 126_458 characters
Saving 148.txt containing 378_600 characters
Saving 158.txt containing 880_424 characters
Saving 161.txt containing 670_674 characters
Saving 171.txt containing 210_070 characters
Saving 300.txt containing 9_597 characters
Saving 317.txt containing 65_908 characters
Saving 409.txt containing 89_156 characters
Saving 468.txt containing 344_623 characters
Saving 554.txt containing 124_162 characters
Saving 566.txt containing 837_577 characters
Saving 574.txt containing 40_051 characters
Saving 577.txt containing 531_468 characters
Saving 601.txt containing 787_145 characters
Saving 652.txt containing 220_039 characters
Saving 696.txt cont

<_io.TextIOWrapper name='/content/drive/MyDrive/Elaborato_finale/txts/1700-1831/76162.txt' mode='wt' encoding='utf-8'>

## 2. Pipeline di *preprocessing*

I file sono quindi pronti nelle rispettive cartelle, ognuna dedicata a un periodo diverso per separare i due corpora su cui allenare i modelli ed effettuare le analisi successive.

Inizialmente, l'idea era quella di trasferire i file TXT all'interno di un altro DataFrame di pandas per conservarli in un unico file CSV che mi permettesse di effettuare tutte le operazioni di pulizia necessarie in maniera comoda e relativamente veloce (già nel Notebook creato per l'esame di Linguaggi di programmazione per l'ambito umanistico (2023/2024) avevo usato come riferimento [questo tutorial](https://programminghistorian.org/en/lessons/corpus-analysis-with-spacy) di Megan S. Kane presente sul sito *Programming Historian* per prendere spunto su come gestire la fase di *preprocessing* di un corpus di testi). Tuttavia, dopo varie prove, ho notato che tale scelta si sarebbe rivelata lenta e, soprattutto, pesante in termini di RAM per Colab. Avevo cercato infatti di rendere più efficiente il lavoro e l'applicazione della *pipeline* seguendo consigli e considerazioni offerti [qui](https://towardsdatascience.com/why-and-how-to-use-pandas-with-large-data-9594dda2ea4c/) e [qui](https://saturncloud.io/blog/reading-large-text-files-with-pandas/) su come gestire grandi quantità di dati e dataset relativamente pesanti, senza però riscontrare grandi risultati.

Ripensandoci a posteriori, avrei potuto provare a vedere se il problema vero non fosse la lunghezza considerevole dei testi a disposizione per i corpora: credo che dividere le opere in estratti più brevi avrebbe potuto migliorare la gestione della RAM, anche se a costo di un processamento più lento.

Ad ogni modo, userò il modulo [glob](https://www.geeksforgeeks.org/how-to-iterate-over-files-in-directory-using-python/) per iterare tra i file delle due cartelle e svolgere la fase di *preprocessing*; la libreria adottata per quest'ultima è stata **spaCy**.

### 2.1 Caricamento e uso dei corpora con pandas e file CSV

Si inizia con alcune osservazioni su quanto si può vedere dai primi file TXT conservati nelle cartelle:



1.   Mentre la pulizia della parte finale dei documenti è risultata perfetta (licenze e informazioni del *PG* sull'uso e distribuzione dei contenuti), non si può dire la stessa cosa su quella all'inizio: spesso rimangono intestazioni e indici, che sono però molto difficili da rimuovere per l'assenza di una struttura standardizzata.
2.   Sarà necessario rimuovere la punteggiatura, che non verrà inclusa tra i *tokens* che saranno resi come vettori nella fase di creazione del modello. Il primo conteggio dei *tokens* (il cui risultato si può consultare al punto 1.2.1) non aveva tenuto conto di fattori come la presenza di composti separati ortograficamente da trattini o nomi propri contenenti l'apostrofo, come quelli di origine irlandese (O'Neill). Essendo i corpora a disposizione particolarmente grandi, ho deciso di non essere puntigliosa e accettare i risultati della tokenizzazione di spaCy (facendo alcune prove con testi più brevi, parole con la stessa forma dei patronimici irlandesi non vengono considerate nella lista finale dei *tokens* puliti, mentre gli [*hyphenated compounds*](https://en.wikipedia.org/wiki/English_compound) vengono divisi nei lessemi corrispondenti).      
3.   Sarà necessario rendere il testo in minuscolo (*lowercasing*) per evitare che il modello si confonda tra forme diverse della stessa parola.
4.   Sarà necessario rimuovere i numeri, non utili alla mia ricerca.
5.   Sarà necessario eliminare le *stopwords*.

Non ho ancora previsto ulteriori azioni da aggiungere alla *pipeline*. Se necessario, operazioni come la lemmatizzazione saranno effettuate dopo aver visto i primi risultati dei modelli. Cercando tra i consigli offerti da utenti dei vari forum specializzati, infatti, mi sono imbattuta in [conversazioni di questo tipo](https://groups.google.com/g/gensim/c/UKW9CTEot-s?pli=1), in cui si consigliava di non lemmatizzare corpora di dimensioni significative (migliaia di righe di dataset e testo dalla lunghezza importante) nel caso in cui si volesse usare Word2Vec. Al momento, non ho trovato conversazioni simili sull'argomento per quel che riguarda l'uso di fastText.

Approfitto della lista di azioni appena creata per definire adesso la funzione di *preprocessing* dedicata all'eliminazione dei segni di punteggiatura, dei numeri, delle *stopwords* e al *lowercasing*. Tali operazioni sono rese possibili, in primis, dalla tokenizzazione, l'unica compresa dal [modello *blank* di spaCy](https://spacy.io/usage/models#quickstart), sulla cui scelta ha influito proprio il fatto di essere provvista dell'unica componente a me necessaria per svolgere i compiti previsti (e, pertanto, di poterli svolgere più rapidamente di una *pipeline* completa). Infine, come si può notare osservando il blocco di codice di caricamento del modello, alla lista di *stopwords* offerta di default da spaCy ho aggiunto altre parole potenzialmente "fastidiose".

In [54]:
def tokenizer_and_basic_cleaner(text):
  tokenized_and_cleaned = []
  if text:
    text = nlp(text)
    for token in text:
      if token.is_alpha and not token.is_stop: #mi servono i tokens alfanumerici che non rientrino nelle stop-words
          tokenized_and_cleaned.append(token.text.lower())


  else:
      print("No text found.")
  return tokenized_and_cleaned

Il passaggio successivo sarà creare un unico grande file TXT aggiungendo di volta in volta (di *token* in *token*) il testo pulito di tutti gli originali. L'idea di fondo è, pertanto, sempre quella di iterare tra i file delle opere, questa volta per creare le versioni "pulite" con la funzione *tokenizer_and_basic_cleaner* e riscrivere queste ultime in un nuovo file TXT in una *directory* diversa.

In [55]:
def all_txts_preprocessing(directory, new_directory, file_name):
  tokens_counter = 0
  complete_new_path = os.path.join(new_directory, file_name)

  isdir = os.path.isdir(directory)

  if isdir:

    for filename in glob.iglob(f'{directory}/*'):

      with open(filename, 'r') as f:
        text = f.read()
        docd_text = tokenizer_and_basic_cleaner(text) #si trasforma il testo in doc e si pulisce dai tokens indesiderati

        with open(complete_new_path, 'a') as file_w_tokens:
          for token in docd_text:
            tokens_counter += 1
            file_w_tokens.write(token + " ")

  else:
    print("Check the directory.")

  return print(f"Total number of tokens for this period: {tokens_counter}.")

In [ ]:
first_period_corpus_directory = "/content/drive/MyDrive/Elaborato_finale/txts/1700-1831" #directory della cartella dei testi da processare

In [ ]:
cleaned_corpus_directory = "/content/drive/MyDrive/Elaborato_finale/txts" #cartella "txt" in cui verrà scritto il file contenente il corpus ripulito token per token

In [ ]:
all_txts_preprocessing(first_period_corpus_directory, cleaned_corpus_directory, "1700-1831.txt")

No text found.
No text found.
No text found.
Total number of tokens for this period: 51827549


In [ ]:
with open(r"/content/drive/MyDrive/Elaborato_finale/txts/1700-1831.txt", "r") as f: #per controllare il risultato
 content = f.read()
 print(content[6000:6500]) #casualmente, tra i token appare pure "verona" :)

ing falieri heroic deeds doria defeated air rang wild shouts triumph additional reason nicolo heaven knows instead going meet doria fleet coolly sailed away returned doria withdrew lagune approach pisani fleet ascribed formidable marino falieri people seignory seized kind frantic ecstasy auspicious choice uncommon way testifying determined welcome newly elected doge messenger heaven bringing honour victory abundance riches nobles accompanied numerous retinue rich dresses sent seignory verona amb


#### 2.1.1 Caricamento e uso file periodo 1832-1932

Ovviamente, la stessa funzione è stata applicata ai testi del secondo corpus:

In [ ]:
second_period_corpus_directory = "/content/drive/MyDrive/Elaborato_finale/txts/1832-1932"

In [ ]:
second_period_files_dataset = all_txts_preprocessing(second_period_corpus_directory, cleaned_corpus_directory, "1832-1932.txt")

No text found.
No text found.
No text found.
No text found.
No text found.
No text found.
Total number of tokens for this period: 40252812


In [ ]:
with open(r"/content/drive/MyDrive/Elaborato_finale/txts/1832-1932.txt", "r") as f: #per controllare il risultato
 content = f.read()
 print(content[2000:3000])

 wonders cluster rotting gambrel roofs bespeaking earlier architectural period neighboring region reassuring closer glance houses deserted falling ruin broken steepled church harbors slovenly mercantile establishment hamlet dreads trust tenebrous tunnel bridge way avoid hard prevent impression faint malign odor village street massed mold decay centuries relief clear place follow narrow road base hills level country till rejoins aylesbury pike afterward learns dunwich outsiders visit dunwich seldom possible certain season horror signboards pointing taken scenery judged ordinary esthetic canon commonly beautiful influx artists summer tourists centuries ago talk witch blood satan worship strange forest presences laughed custom reasons avoiding locality sensible age dunwich horror hushed town world welfare heart people shun knowing exactly reason apply uninformed strangers natives repellently decadent having gone far path retrogression common new england backwaters come form race defined m

Da quello che si può vedere dal nuovo conteggio di *tokens*, rimane una certa differenza quantitativa, seppur di molto ridotta, tra i due corpora. Se ciò si possa percepire una volta che i modelli sono stati creati si vedrà nella successiva fase di allenamento e *testing* dei modelli, per la quale si rimanda al secondo notebook che accompagna questo progetto.

